<a href="https://colab.research.google.com/github/btatsis/learn2smell/blob/master/learn2smell.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [95]:
# Taken from AIcrowd github repo
#!git clone https://github.com/AIcrowd/learning-to-smell-starter-kit
#!cd learning-to-smell-starter-kit
#!pip install -r requirements.txt

In [54]:
!mkdir data
!git clone https://github.com/btatsis/learn2smell.git  .data/

Cloning into '.data'...
remote: Enumerating objects: 13, done.
remote: Counting objects: 100% (13/13), done.
remote: Compressing objects: 100% (12/12), done.
remote: Total 13 (delta 2), reused 0 (delta 0), pack-reused 0
Unpacking objects: 100% (13/13), done.


In [96]:
df = pd.read_csv('.data/train.csv', sep='^([^,]+),', engine='python', usecols=['SMILES', 'SENTENCE'])
df.rename({'SMILES': 'smiles', 'SENTENCE': 'smell'}, axis=1, inplace=True)
df.smell = df.smell.str.replace('"', '')

df = pd.concat([df, df.smell.str.get_dummies(sep=',')],axis=1)
df.head(3)

                        smiles                  smell
0  C/C=C/C(=O)C1CCC(C=C1C)(C)C            fruity,rose
1                    COC(=O)OC  fresh,ethereal,fruity
2          Cc1cc2c([nH]1)cccc2      resinous,animalic


,smiles,smell,alcoholic,aldehydic,alliaceous,almond,ambergris,ambery,ambrette,ammoniac,animalic,anisic,apple,balsamic,banana,berry,blackcurrant,blueberry,body,bread,burnt,butter,cacao,camphor,caramellic,cedar,cheese,chemical,cherry,cinnamon,citrus,clean,clove,coconut,coffee,cognac,coniferous,cooked,cooling,cucumber,...,mint,mushroom,musk,musty,nut,odorless,oily,orange,overripe,pear,pepper,phenolic,plastic,plum,powdery,pungent,rancid,resinous,ripe,roasted,rose,seafood,sharp,smoky,sour,spicy,sulfuric,sweet,syrup,terpenic,tobacco,tropicalfruit,vanilla,vegetable,violetflower,watery,waxy,whiteflower,wine,woody
0,C/C=C/C(=O)C1CCC(C=C1C)(C)C,"fruity,rose",0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
1,COC(=O)OC,"fresh,ethereal,fruity",0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
2,Cc1cc2c([nH]1)cccc2,"resinous,animalic",0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0


In [103]:
import numpy as np
labels = np.array(df.iloc[:,2:])

print(labels.shape)

(4316, 109)


In [105]:
!wget -c https://repo.continuum.io/miniconda/Miniconda3-py37_4.8.3-Linux-x86_64.sh
!chmod +x Miniconda3-py37_4.8.3-Linux-x86_64.sh
!time bash ./Miniconda3-py37_4.8.3-Linux-x86_64.sh -b -f -p /usr/local
!time conda install -q -y -c conda-forge rdkit

import sys
sys.path.append('/usr/local/lib/python3.7/site-packages/')

--2020-09-09 21:36:23--  https://repo.continuum.io/miniconda/Miniconda3-py37_4.8.3-Linux-x86_64.sh
Resolving repo.continuum.io (repo.continuum.io)... 104.18.201.79, 104.18.200.79, 2606:4700::6812:c84f, ...
Connecting to repo.continuum.io (repo.continuum.io)|104.18.201.79|:443... connected.
HTTP request sent, awaiting response... 301 Moved Permanently
Location: https://repo.anaconda.com/miniconda/Miniconda3-py37_4.8.3-Linux-x86_64.sh [following]
--2020-09-09 21:36:23--  https://repo.anaconda.com/miniconda/Miniconda3-py37_4.8.3-Linux-x86_64.sh
Resolving repo.anaconda.com (repo.anaconda.com)... 104.16.130.3, 104.16.131.3, 2606:4700::6810:8303, ...
Connecting to repo.anaconda.com (repo.anaconda.com)|104.16.130.3|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 88867207 (85M) [application/x-sh]
Saving to: ‘Miniconda3-py37_4.8.3-Linux-x86_64.sh’

Miniconda3-py37_4.8 100%[===================>]  84.75M   153MB/s    in 0.6s    

2020-09-09 21:36:24 (153 MB/s) - ‘Minicon

In [121]:
import rdkit 
print(rdkit.__version__)

from rdkit import Chem, DataStructs
from rdkit.Chem import AllChem

from rdkit.Chem import Draw
from rdkit.Chem.Draw import IPythonConsole
from rdkit.Chem import PandasTools

2020.03.5


In [115]:
PandasTools.AddMoleculeColumnToFrame(df, smilesCol='smiles')
df.head(1)

fps = [AllChem.GetMorganFingerprintAsBitVect(mol,radius=3, nBits=2048) for mol in df['ROMol']]
X = np.array(list(fps))
print(X.shape)

(4316, 2048)


In [126]:
from tqdm import tqdm

def fp_list_from_smiles_list(smiles_list):
  fp_list = []
  for smiles in tqdm(smiles_list):
    mol = Chem.MolFromSmiles(smiles)
    fp_list.append(fp_as_array(mol))
  return fp_list

def fp_as_array(mol):
  fp = AllChem.GetMorganFingerprintAsBitVect(mol, 2)
  arr = np.zeros((1,), np.int)
  DataStructs.ConvertToNumpyArray(fp, arr)
  return arr

X = fp_list_from_smiles_list(df.smiles) 

100%|██████████| 4316/4316 [00:00<00:00, 5775.73it/s]


In [ ]:
from sklearn.datasets import make_classification
from sklearn.multioutput import MultiOutputClassifier
from sklearn.ensemble import RandomForestClassifier

rf_model = RandomForestClassifier(random_state=1)
multi_target_forest = MultiOutputClassifier(rf_model, n_jobs=-1)
multi_target_forest.fit(X, labels).predict(X)

In [ ]:
df_test = pd.read_csv('.data/test.csv', sep='^([^,]+),', engine='python', usecols=['SMILES', 'SENTENCE'])
df.rename({'SMILES': 'smiles', 'SENTENCE': 'smell'}, axis=1, inplace=True)
df.smell = df.smell.str.replace('"', '')

df = pd.concat([df, df.smell.str.get_dummies(sep=',')],axis=1)
df.head(3)

## Script for results submission in AIcrowd competition

In [ ]:
#!/usr/bin/env python

import pandas as pd
import json

import os
import random


TRAINING_DATASET_PATH = os.getenv("TRAINING_DATASET_PATH", "data/train.csv")
TEST_DATASET_PATH = os.getenv("TEST_DATASET_PATH", "data/test.csv")

VOCABULARY_PATH = os.getenv("VOCABULARY_PATH", "data/vocabulary.txt")


vocabulary = open(VOCABULARY_PATH).read().split()


test_df = pd.read_csv(TEST_DATASET_PATH)


def generate_sample_submission(df, vocabulary):
    """
    Generates a sample submission file
    by randomly generating 5 random smell sentences 
    of upto 3 random smell words.

    The 5 sentences are to be separated by a ;
    and all the individual smells words in 
    a sentence are to be separated by a ,
    """

    def generate_random_prediction():
        # Generates a single prediction string

        SENTENCES = []
        for _ in range(5):
            WORDS = []
            for _ in range(3):
                WORDS.append(
                    random.choice(vocabulary)
                )
            SENTENCES.append(",".join(sorted(set(WORDS))))
        return ";".join(SENTENCES)
    
    predictions = [generate_random_prediction() for _ in range(df.shape[0])]

    return pd.DataFrame({
        "SMILES" : df.SMILES.tolist(),
        "PREDICTIONS" : predictions
    })

sample_submission_df = generate_sample_submission(test_df, vocabulary)

submission_file_path = "submission.csv"
print("Writing Sample Submission to : ", submission_file_path)
sample_submission_df.to_csv(
    submission_file_path,
    index=False
)